In [4]:
from eventregistry import *
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions
import json
import pandas as pd
from newspaper import Article

In [25]:
global_df = pd.DataFrame()
eventregistry_api = 'aa1174a5-a633-492d-a182-28d59fc28a34'
watson_api = 'VDaX2sFbNmoKB3gQit67DgC_2y5hnsEwLA1RU49Ko65H'
bodies_df = pd.DataFrame()

def getInputArticleKeywords(user_url):
#     url = user_url.decode('utf-8')
    article = Article(user_url)
    article.download()
    article.parse()
    article.nlp()

    keywords = article.keywords
    kws = []
    for word in keywords:
        if len(kws) < 10:
            kws.append(word)

    return kws


def getArticles(keywords):
    global global_df
    global bodies_df
#     bodies_df = pd.DataFrame()
    
    er = EventRegistry(apiKey = eventregistry_api)
    q = QueryArticlesIter(
        keywords = QueryItems.OR(keywords))
        # keywordsLoc = "title",
        # ignoreKeywords = "SpaceX",
        # sourceUri = "nytimes.com")
    q.setRequestedResult(RequestArticlesInfo(sortBy="rel"))
    res = q.execQuery(er, sortBy = "rel", maxItems = 500)
    local_df = pd.DataFrame()
    local_body_df = pd.DataFrame()
    index = 0
    body_count = 0
    
#     print(body_count)
    for article in res:
        data = {
            'source': article['source']['title'].encode('utf-8'),
            'url' : article['url'].encode('utf-8'),
            'text' : article['body'].encode('utf-8')
        }
        
        body = {
            'articleBody': article['body'],
            'Body ID': body_count
        }
        
        local_body_df = pd.concat([local_body_df, pd.DataFrame(body, index=[index])])
        local_df = pd.concat([local_df, pd.DataFrame(data,index=[index])])
#         index += 1
        body_count += 1
    
    # append to global dataframe
    global_df = pd.concat([global_df,local_df])
    bodies_df = pd.concat([bodies_df, local_body_df])
    
def getKeywords(url):
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2018-11-16',
        iam_apikey=watson_api,
        url='https://gateway-lon.watsonplatform.net/natural-language-understanding/api'
    )

    response = natural_language_understanding.analyze(
        url=url,
        features=Features(keywords=KeywordsOptions())).get_result()

    keywords = []
    
    for keyword in response['keywords']:
        if keyword['relevance'] > 0.8 and len(keywords) < 10:
            keywords.append(keyword['text'].encode('utf-8'))
    return keywords
    
# given the url of 
def webscrapeMain(url):
    global global_df
    # get keywords 
    kws = getInputArticleKeywords(url)
#     kws = getKeywords(url)
    print("The keywords are: {}".format(kws))

    getArticles(kws)
    print(bodies_df)
    global_df = global_df.reset_index(drop=True)
    global_df.to_csv('articles.csv')
    
   
    bodies_df.to_csv('bodies.csv')
    
# run
url = 'https://www.washingtonpost.com/news/powerpost/wp/2018/01/11/joe-arpaio-is-back-and-brought-his-undying-obama-birther-theory-with-him/?utm_term=.6e970c87deba'
webscrapeMain(url)
print("Finished scraping for articles related to the article")


The keywords are: ['obamas', 'obama', 'president', 'arpaio', 'birth', 'certificate', 'theory', 'campaign', 'brought', 'joe']
using user provided API key for making requests
Event Registry host: http://eventregistry.org
Text analytics host: http://analytics.eventregistry.org
                                          articleBody  Body ID
0   Scientific Evidence: Obama Birth Certificate C...        0
0   For the fourth year in a row, Donald Trump dom...        1
0   The Democratic Party is screwing up its opposi...        2
0   Former President Obama's talk about how politi...        3
0   Right-wingers long to expose Alexandria Ocasio...        4
0   The 2020 presidential race is shaping up on th...        5
0   Marriages: Melania (Knauss) Trump (January 22,...        6
0   Marriages: Melania (Knauss) Trump (January 22,...        7
0   By Emily Goodin, U.s. Political Reporter For D...        8
0   At noon on Jan. 3, the 435 members of the Hous...        9
0   Marriages: Melania (Knauss) 

In [12]:
import nltk
import requests
from IPython.display import display, HTML

azure_pd = pd.DataFrame()
azure_key = '34d4fdab594e46c2b8f4b497042a7260'
search_url = "https://api.cognitive.microsoft.com/bing/v7.0/search"


def azureSearch(claim):
    headers = {"Ocp-Apim-Subscription-Key" : azure_key}
    params  = {"q": claim, "textDecorations":True, "textFormat":"HTML"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"],v["name"],v["snippet"]) \
                  for v in search_results["webPages"]["value"]])
#     display(HTML("<table>{0}</table>".format(rows)))
    
    for article in search_results["webPages"]["value"]:
#         if index == 0:
        webscrapeMain(article['url'])
#         print(json.dumps(article, indent=4))
        break
#         data = {
#             'source': article['source'].encode('utf-8'),
#             'url' : article['url'].encode('utf-8'),
#             'text' : article['body'].encode('utf-8')
#         }
#         local_df = pd.concat([local_df, pd.DataFrame(data,index=[index])])
#         index += 1
    
    # append to global dataframe
#     azure_pd = pd.concat([azure_pd,local_df])

claim = 'Obama is not American'
azureSearch(claim)

{
    "id": "https://api.cognitive.microsoft.com/api/v7/#WebPages.0",
    "name": "Barack Obama citizenship conspiracy theories - Wikipedia",
    "url": "https://en.wikipedia.org/wiki/Barack_Obama_citizenship_conspiracy_theories",
    "about": [
        {
            "name": "Barack Obama citizenship conspiracy theories"
        }
    ],
    "isFamilyFriendly": true,
    "displayUrl": "https://<b>en.wikipedia.org</b>/wiki/<b>Barack_Obama</b>_citizenship_conspiracy...",
    "snippet": "Williams asked <b>Obama</b> about the fact that a fifth of the <b>American</b> people do <b>not</b> believe that he is either <b>American</b> born or a Christian. <b>Obama</b> responded that &quot;there is a mechanism, a network of misinformation that in a new media era can get churned out there constantly&quot;.",
    "snippetAttribution": {
        "license": {
            "name": "CC-BY-SA",
            "url": "http://creativecommons.org/licenses/by-sa/3.0/"
        },
        "licenseNotice": "\u05d8\

In [20]:
def generateClaimCSV(claim):
    data = pd.read_csv("articles.csv")
    out = pd.DataFrame()
    
    claims = [claim] * len(data.index)
    
#     claims = pd.DataFrame(claim)
    out['id'] = range(len(data.index))
    out['Headline'] = claims
    out['Body ID'] = range(len(data.index))
    out.to_csv('claims.csv')
    print("Succesfully generated claims")

claim = 'Obama is not American'
generateClaimCSV(claim)

Succesfully generated claims
